In [1]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
import urllib.request
import os
import urllib.parse
import shutil
from io import BytesIO

#import cv2
from PIL import Image
import time
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np


In [2]:
fakeBlogKeywordList = ["스토리앤","seoulouba","mateb.kr","revu","weble","ohmyblog","mrblog","tble",
    "dinnerqueen","%EA%B3%B5%EC%A0%95%EA%B1%B0%EB%9E%98%EC%9C%84%EC%9B%90%ED%9A%8C-%EB%AC%B8%EA%B5%AC",
    "%EB%B8%94%EB%A1%9C%EA%B7%B8%EC%9B%90%EC%A0%95%EB%8C%80",
    "banner_","%EC%9D%B4%EC%8A%88%EB%B8%94%EB%A1%9C%EA%B7%B8",
    "%EB%B0%B0%EB%84%88","http://echoblog.net/images/sponsor-banner.png",
                      "sponsor","banner","echoblog"]

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

#define const value
batch_size = 3000
epochs = 15
IMG_HEIGHT = 260
IMG_WIDTH = 460


In [4]:
def create_model():
    model = Sequential([
        Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
        MaxPooling2D(),
        Conv2D(32, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Conv2D(64, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [5]:
#load model
checkpoint_path = "model_diff_width_0707_v1(batch-24)/cp-0015.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

# create model
model = create_model()
model.load_weights(latest)

data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
np.set_printoptions(suppress=True)

model_diff_width_0707_v1(batch-24)/cp-0015.ckpt


In [6]:
def convertToJpeg(im):
    with BytesIO() as f:
        new_img = im.convert("RGB")
        new_img.save(f, format='JPEG')
        return f.getvalue()


In [7]:
def PredictionWithoutDownload(uri):

    start_time = time.time();
    filename = uri.split('/')[-1].split('?')[0];
    img_url2 = urllib.parse.quote_plus(str(filename))
    img_url_final = uri.replace(uri.split('/')[-1].split('?')[0],img_url2)
    
#     response = requests.get(uri)
    response = urllib.request.urlopen(img_url_final).read()
#     print("---[1] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    #image = tf.keras.preprocessing.image.load_img(image_path,target_size=(IMG_HEIGHT, IMG_WIDTH))
#     img = Image.open(BytesIO(response.content))
    img = Image.open(BytesIO(response))
    
    if(img.format == "PNG") or (img.format == "GIF"):
        print("this image is",img.format,"type. could be change to jpg")
        png_to_jpg = convertToJpeg(img)
        img = Image.open(BytesIO(png_to_jpg))
#         img.show()
#     print("---[2] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    #width,height
    resize_img = img.resize((IMG_WIDTH, IMG_HEIGHT))
#     print("---[3] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();

    input_arr = tf.keras.preprocessing.image.img_to_array(resize_img)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
#     print("---[4] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    
    predictions_class = model.predict_classes(input_arr)
#     print("---[5] {}s seconds---".format((time.time()-start_time),".2f"))

    print("========pridiction[Without Download]========");
#     print(test_data_gen.class_indices)
    print("predictions class : " , predictions_class);
    if(predictions_class == [[0]]):
        predictions = model.predict(input_arr)
        print("predictions : ",predictions);

        print("this is fake blog")
        saveImageVol2(uri,predictions)
#     print("predictions : " , predictions)
    return predictions_class


In [8]:
# =====================================================================
#이미지 저장만을 위한 함수
def saveImageVol2(img_url,predict):
    list = os.listdir("ad_img")
    last_index = len(list)
#         print("last_index : ",last_index)
    dirPath = 'ad_img'
    imgPath = f'ad_img/{last_index}_{predict}.jpg'
    
    if not os.path.isdir(dirPath):
        #폴더 생성
        try:
            os.makedirs(dirPath)
        except Exception as ex:
            print("error catch : " , ex)
    
    try:
        filename = img_url.split('/')[-1].split('?')[0];
        img_url2 = urllib.parse.quote_plus(str(filename))
        img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)
        urllib.request.urlretrieve(img_url_final, imgPath)
        #분류
        #{'advertise': 0, 'image': 1}
        #my_prediction  = teachableMachinePrediction(imgPath);
#         print("prediction : ", my_prediction[0]);
    except Exception as ex:
        print("img save error : " , ex)
        return False

In [33]:
# =====================================================================
#이미지 분류 [my model]

def imageClassification(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path,target_size=(IMG_HEIGHT, IMG_WIDTH))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
#    print("input_arr type : " ,type(input_arr))
    input_arr = np.array([input_arr])  # Convert single image to a batch.

    predictions_class = model.predict_classes(input_arr)
    predictions = model.predict(input_arr)

    print("========pridiction========");
#     print(test_data_gen.class_indices)
    print("predictions class : " , predictions_class);
    print("predictions : " , predictions);
#    print("predictions argmax : ",np.argmax(predictions[0])); # same way
    return predictions_class,predictions;

# =====================================================================
#이미지 저장
def saveImage(div_option,index,img_url,nickName,postNumber):
    #블로그 유저 별로 폴더를 나눌건지에 대한 옵션
    #true -> path : image/{nickName}/{postNumber}
    #false -> path : image/myimg/
    if(div_option):
        if nickName is "":
            nickName = "unknown"
        if postNumber is "":
            postNumber = "1"        
        imgPath = f'image/{nickName}/{postNumber}/{index}.jpg'
        dirPath = f'image/{nickName}/{postNumber}'
    #폴더별로 분기하지 않을떄 path : image/myimg/
    else:
        list = os.listdir("image/temp")
        last_index = len(list)
#         print("last_index : ",last_index)
        dirPath = 'image/temp'
        imgPath = f'image/temp/{last_index}.jpg'
    
    if not os.path.isdir(dirPath):
        #폴더 생성
        try:
            os.makedirs(dirPath)
        except Exception as ex:
            print("error catch : " , ex)
                
    #파일 저장
    #temp에 저장 -> classification -> 분류 저장
    
    try:
        filename = img_url.split('/')[-1].split('?')[0];
        img_url2 = urllib.parse.quote_plus(str(filename))
        img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)
#         print("img_url_final : " ,img_url_final);
        for item in fakeBlogKeywordList:
            if(img_url_final.find(item) > 0):
                print('this is fake img')

                imgPath = f"image/temp/fake_{last_index}.jpg"
                break;
        
        urllib.request.urlretrieve(img_url_final, imgPath)
        #분류
        #{'advertise': 0, 'image': 1}
        my_prediction, score  = imageClassification(imgPath);
        #my_prediction  = teachableMachinePrediction(imgPath);
#         print("prediction : ", my_prediction[0]);
        
        if (my_prediction[0] == 0):
            list = os.listdir("ad_img")
            last_index = len(list)
            #os.rename(imgPath, f'ad_img/{last_index}.jpg')
            #shutil.move(imgPath, f'ad_img/{last_index}.jpg')
            if(imgPath.find("fake_") > 0):  #모델, url 둘다 이미지로 판단
                os.replace(imgPath, f'ad_img/url_model_{last_index}_{score}.jpg')
            else: #모델만 이미지로 판단
                os.replace(imgPath, f'ad_img/model_{last_index}_{score}.jpg')
            return False
        else: # url만 이미지로 판단
            list = os.listdir("not_fake")
            last_index = len(list)
            if(imgPath.find("fake_") > 0):
                list = os.listdir("ad_img")
                last_index = len(list)
                os.replace(imgPath, f'not_fake/url_{last_index}_{score}.jpg')
            else: #모델, url 둘다 이미지로 판단
                if(score <= 1500):  #confidence가 1500이하일때만 저장 
                    os.replace(imgPath, f'not_fake/url_model_{last_index}_{score}.jpg')
        return True
    except Exception as ex:
        print("img save error : " , ex)
        return False
    
    
# =====================================================================
# image gethering
def findBlogImgGethering(searchQuery, searchOption, page):
    try:
        start_time = time.time();

        url = f'https://search.naver.com/search.naver?query={searchQuery}&sm=tab_pge&srchby=all&st={searchOption}&where=post&start={page}'
        html = requests.get(url)
        # 1차, blog URL 찾기
        soup = BeautifulSoup(html.text, 'html.parser')
        index = 0;

        url_list = [];
        for li_item in soup.find_all('li',{'class' : 'sh_blog_top'}):
            child_item = li_item.find('a',{'class' : 'sh_blog_title'})
            title = child_item.attrs['title']
            href = child_item.attrs['href']

            #url list 저장
            url_list.append(href)

            print(f'title : {title} , href : {href}')
            print('----------------------------------')

        # url 파싱 및 예외처리
        for url_item in url_list:
            nickName = "";
            postNumber = "";

            if(url_item.find("blog.me") > 0):
                parsing = url_item;
                parsing = parsing.replace("https://","")
                nickName = parsing.split('.')[0]
                postNumber = parsing.split('/')[1]
                blogUrl = "https://m.blog.naver.com/" + nickName + "?Redirect=Log&logNo=" + postNumber

            else:
                blogUrl = url_item.replace("https://", "https://m.");

            nickName = blogUrl.split('/')[-1].split('?')[0]
            postNumber = blogUrl.split('/')[-1].split('?')[1].split('=')[-1]

            if nickName is "":
                nickName = "unknown"
            if postNumber is "":
                postNumber = "1"

            print("------ digging more -------")
            print("nickName : " , blogUrl.split('/')[-1].split('?')[0])
            print("postNumber : " , blogUrl.split('/')[-1].split('?')[1].split('=')[-1])
            print("blogUrl : " , blogUrl)        
            start_time = time.time();
            #전체 목록 순회
            blog_html = requests.get(blogUrl)
            blog_soup = BeautifulSoup(blog_html.text, 'html.parser')
            blog_image_class = blog_soup.find_all('div',{'class' : 'se-image'});

            for div_obj in blog_image_class:
                for idx, img_item in enumerate(div_obj.find_all('img')):
                    img_url = img_item.attrs['src'];

                    if(img_url.find("w80_blur") > 0):
                        img_url = img_url.replace("w80_blur" , "w800")
                        #np array after save image(for crawling data)
                    isSaveSuccess = saveImage(False,index,img_url,nickName,postNumber)
                    #np arr from img_url for validation
#                     PredictionWithoutDownload(img_url)
                    if(isSaveSuccess):  
                        index = index+1
    #         print("---{}s seconds---".format(time.time()-start_time),".2f")
    except Exception as ex:
        print("error catch : " , ex)
        print("url : " ,img_url)


In [ ]:
searchQuery = "서울대 피자집"
searchOption = 'sim'  #sim or date  
page = 1
end_page = 10
for i in list(range(page,end_page)):
    if i>1:
        page = page + 10;
    findBlogImgGethering(searchQuery,searchOption, page)
# findBlogImgGethering(searchQuery,searchOption,page)

title : 서울대입구역 피자 샤로수길 갓성비 피자집 피자보이시나 , href : https://blog.naver.com/hs486mh?Redirect=Log&logNo=221683003524
----------------------------------
title : 서울대입구역 샤로수길 데이트 : 독립서점 살롱드북, 피자집 파고, 카페 미드레벨 , href : https://blog.naver.com/areum_land?Redirect=Log&logNo=221880263306
----------------------------------
title : 서울대입구역 피자, 피자보이시나(샤로수길 피자집) 맛보시게나 (메뉴&가격) , href : https://blog.naver.com/ji_ram_jiral?Redirect=Log&logNo=222018690249
----------------------------------
title : 서울대입구 데이트하기 좋은 아기자기한 피자집 ‘이태리파파’ , href : https://blog.naver.com/somethinglikethat?Redirect=Log&logNo=221762850715
----------------------------------
title : 서울대입구역 피자집 : 피자네버슬립스 ☆ , href : https://blog.naver.com/1john324?Redirect=Log&logNo=221707130754
----------------------------------
title : 서울대입구역, 샤로수길 피자집을 찾았다! (tailor slice pizza) , href : https://blog.naver.com/7474562?Redirect=Log&logNo=221385130551
----------------------------------
title : 서울대입구 피자 피자네버슬립스 샤로수길점은 피자집 1등이라죠 ?? , href : https://blog.naver.c

========pridiction========
predictions class :  [[1]]
predictions :  [[2788.9866]]
========pridiction========
predictions class :  [[1]]
predictions :  [[5746.014]]
------ digging more -------
nickName :  1john324
postNumber :  221707130754
blogUrl :  https://m.blog.naver.com/1john324?Redirect=Log&logNo=221707130754
========pridiction========
predictions class :  [[1]]
predictions :  [[6429.6606]]
========pridiction========
predictions class :  [[1]]
predictions :  [[6974.474]]
========pridiction========
predictions class :  [[1]]
predictions :  [[8527.858]]
========pridiction========
predictions class :  [[1]]
predictions :  [[12456.484]]
========pridiction========
predictions class :  [[1]]
predictions :  [[12137.257]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9431.83]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9081.392]]
========pridiction========
predictions class :  [[1]]
predictions :  [[13701.8545]]
========pridiction====

========pridiction========
predictions class :  [[1]]
predictions :  [[15389.519]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9050.297]]
========pridiction========
predictions class :  [[1]]
predictions :  [[4608.1304]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2729.202]]
========pridiction========
predictions class :  [[1]]
predictions :  [[10421.947]]
========pridiction========
predictions class :  [[1]]
predictions :  [[8108.8574]]
------ digging more -------
nickName :  a878062
postNumber :  221121999261
blogUrl :  https://m.blog.naver.com/a878062?Redirect=Log&logNo=221121999261
------ digging more -------
nickName :  haanoon
postNumber :  220513662755
blogUrl :  https://m.blog.naver.com/haanoon?Redirect=Log&logNo=220513662755
------ digging more -------
nickName :  wino_g
postNumber :  220548091419
blogUrl :  https://m.blog.naver.com/wino_g?Redirect=Log&logNo=220548091419
------ digging more -------
nickName :  rbgks516
post

========pridiction========
predictions class :  [[1]]
predictions :  [[3045.3813]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-968.71545]]
========pridiction========
predictions class :  [[1]]
predictions :  [[8371.915]]
========pridiction========
predictions class :  [[1]]
predictions :  [[5486.2183]]
this is fake img
========pridiction========
predictions class :  [[0]]
predictions :  [[-10377.865]]
------ digging more -------
nickName :  starry_life
postNumber :  221656009732
blogUrl :  https://m.blog.naver.com/starry_life?Redirect=Log&logNo=221656009732
========pridiction========
predictions class :  [[1]]
predictions :  [[11230.504]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1986.5386]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2873.7258]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-71.69455]]
========pridiction========
predictions class :  [[1]]
predictions :  [[5000.

========pridiction========
predictions class :  [[1]]
predictions :  [[1451.7063]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1507.7839]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1780.8689]]
========pridiction========
predictions class :  [[1]]
predictions :  [[10910.671]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-4112.5835]]
========pridiction========
predictions class :  [[1]]
predictions :  [[4765.4775]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2931.5002]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1942.5643]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9478.635]]
========pridiction========
predictions class :  [[1]]
predictions :  [[10860.819]]
========pridiction========
predictions class :  [[1]]
predictions :  [[7765.4673]]
========pridiction========
predictions class :  [[1]]
predictions :  [[7909.7554]]
===

...

========pridiction========
predictions class :  [[1]]
predictions :  [[11855.115]]
========pridiction========
predictions class :  [[1]]
predictions :  [[8329.575]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9525.474]]
========pridiction========
predictions class :  [[1]]
predictions :  [[5204.459]]
========pridiction========
predictions class :  [[1]]
predictions :  [[5674.5635]]
========pridiction========
predictions class :  [[1]]
predictions :  [[12784.293]]
========pridiction========
predictions class :  [[1]]
predictions :  [[12295.305]]
========pridiction========
predictions class :  [[1]]
predictions :  [[10401.068]]
========pridiction========
predictions class :  [[1]]
predictions :  [[10000.151]]
========pridiction========
predictions class :  [[1]]
predictions :  [[12430.821]]
========pridiction========
predictions class :  [[1]]
predictions :  [[14619.612]]
========pridiction========
predictions class :  [[1]]
predictions :  [[11803.123]]
=======

========pridiction========
predictions class :  [[1]]
predictions :  [[14560.986]]
========pridiction========
predictions class :  [[1]]
predictions :  [[15809.608]]
========pridiction========
predictions class :  [[1]]
predictions :  [[7958.4653]]
========pridiction========
predictions class :  [[1]]
predictions :  [[10075.068]]
========pridiction========
predictions class :  [[1]]
predictions :  [[12143.028]]
========pridiction========
predictions class :  [[1]]
predictions :  [[7713.7393]]
========pridiction========
predictions class :  [[1]]
predictions :  [[14411.763]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-5516.655]]
error catch :  list index out of range
url :  https://mblogthumb-phinf.pstatic.net/MjAxOTExMjBfMjgz/MDAxNTc0MjYwNDg2OTU1.doov1mrW1BSlgfYdjHIV5Qjs45J9bEcj9Z0DgtXHNJ4g.rx23VQW8zv2yGesLMzely4rxLyJOsP9Q8yCCCR-Dx4sg.PNG.byvividb/IMG_4334.PNG?type=w800
title : 서울대입구 샤로수길 피자 - 돌쇠아저씨 화덕피자에서 고르곤졸라 피자와 생맥주, 불금 피맥파티~ , href : https://blog.naver.

========pridiction========
predictions class :  [[1]]
predictions :  [[11503.84]]
========pridiction========
predictions class :  [[1]]
predictions :  [[11106.287]]
========pridiction========
predictions class :  [[1]]
predictions :  [[12980.687]]
========pridiction========
predictions class :  [[1]]
predictions :  [[8465.587]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9262.033]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9229.023]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9987.86]]
========pridiction========
predictions class :  [[1]]
predictions :  [[6814.7837]]
========pridiction========
predictions class :  [[1]]
predictions :  [[7759.115]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3290.0396]]
this is fake img
========pridiction========
predictions class :  [[0]]
predictions :  [[-5516.655]]
------ digging more -------
nickName :  seoul365clinic
postNumber :  22145777

========pridiction========
predictions class :  [[1]]
predictions :  [[7472.029]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9657.369]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9530.627]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9457.078]]
========pridiction========
predictions class :  [[1]]
predictions :  [[10291.209]]
========pridiction========
predictions class :  [[1]]
predictions :  [[7429.546]]
========pridiction========
predictions class :  [[1]]
predictions :  [[6573.3984]]
========pridiction========
predictions class :  [[1]]
predictions :  [[17797.592]]
========pridiction========
predictions class :  [[1]]
predictions :  [[13355.111]]
========pridiction========
predictions class :  [[1]]
predictions :  [[7708.3716]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3682.4897]]
========pridiction========
predictions class :  [[1]]
predictions :  [[8789.089]]
========pr

========pridiction========
predictions class :  [[1]]
predictions :  [[3986.0176]]
========pridiction========
predictions class :  [[1]]
predictions :  [[6783.5547]]
========pridiction========
predictions class :  [[1]]
predictions :  [[8743.775]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9345.948]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2894.837]]
========pridiction========
predictions class :  [[1]]
predictions :  [[9495.169]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3899.9868]]
========pridiction========
predictions class :  [[1]]
predictions :  [[7039.226]]
========pridiction========
predictions class :  [[1]]
predictions :  [[7838.9893]]
========pridiction========
predictions class :  [[1]]
predictions :  [[4837.8228]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2254.1118]]
========pridiction========
predictions class :  [[1]]
predictions :  [[832.6424]]
========pr

In [29]:
# blog gethering test
def findBlogListTest(searchQuery, searchOption, page):
    try:
        print(f'\n###############{page}###############\n');
        url = f'https://search.naver.com/search.naver?query={searchQuery}&sm=tab_pge&srchby=all&st={searchOption}&where=post&start={page}'
        html = requests.get(url)
        # 1차, blog URL 찾기
        soup = BeautifulSoup(html.text, 'html.parser')
        index = 0;

        url_list = [];
        for li_item in soup.find_all('li',{'class' : 'sh_blog_top'}):
            child_item = li_item.find('a',{'class' : 'sh_blog_title'})
            title = child_item.attrs['title']
            href = child_item.attrs['href']

            #url list 저장
            url_list.append(href)

            print(f'title : {title} , href : {href}')
            print('----------------------------------')

    except Exception as ex:
        print("error catch : " , ex)
        print("url : " ,img_url)

In [31]:
searchQuery = "서울대입구역 하남돼지집"
searchOption = 'sim'  #sim or date  
page = 1
end_page = 11
# 1->11->21->순으로
for i in list(range(page,end_page)):
    if i>1:
        page = page + 10;
    findBlogListTest(searchQuery,searchOption,page)


###############1###############

title : 서울대입구역 고기집_하남돼지집 , href : https://blog.naver.com/smilly07?Redirect=Log&logNo=221986387337
----------------------------------
title : [서울대입구역] 하남돼지집 - 서울대입구역 삼겹살 , href : https://blog.naver.com/juhun000?Redirect=Log&logNo=221944015147
----------------------------------
title : 서울대입구역 삼겹살은 역시 하남돼지집 , href : https://blog.naver.com/dbwlsdbdus?Redirect=Log&logNo=221984816209
----------------------------------
title : 서울대입구역 맛집 손도 입도 편한 고기꿀맛집 하남돼지집 , href : https://blog.naver.com/clmusicdi?Redirect=Log&logNo=221834473267
----------------------------------
title : 서울대입구역 삼겹살 - 하남돼지집 서울대입구점 , href : https://rich4707.blog.me/221479303167
----------------------------------
title : 서울대입구역 삼겹살 맛집 : 하남돼지집 , href : https://maxsunao.blog.me/221443369975
----------------------------------
title : 서울대입구역 고기집 [하남돼지집] , href : https://soomin4745.blog.me/222014347932
----------------------------------
title : 서울대입구역 맛집 <하남돼지집> 영국남자처럼 우리도 정복! , href : https://blog.

title : 서울대입구역 하남돼지집 고기 맛집 , href : https://blog.naver.com/clsrn361?Redirect=Log&logNo=221927945000
----------------------------------
title : [맛집] 고기 재질 걍 미쳤음; 서울대입구역 맛집 존맛탱 하남돼지집 , href : https://blog.naver.com/qnwkalswn?Redirect=Log&logNo=221747288132
----------------------------------
title : ♥️서울대입구역 고기집 하남돼지집 삼겹살이 예술이네 , href : https://blog.naver.com/hmlee68?Redirect=Log&logNo=221440289267
----------------------------------
title : 샤로수길 하남돼지집/서울대입구역 삼겹살 맛집 하남돼지집 , href : https://blog.naver.com/zkxmqkek?Redirect=Log&logNo=221777565227
----------------------------------
title : 서울대입구역 고기집 하남돼지집에서 육즙가득한 모임 , href : https://blog.naver.com/purplecrom?Redirect=Log&logNo=221333173557
----------------------------------
title : [서울대입구역 : 하남돼지집] 맛있는 한돈을 먹을 수 있는 곳 (내 돈, 메뉴O) , href : https://blog.naver.com/hyund4705?Redirect=Log&logNo=221926433064
----------------------------------
title : [관악구/서울대입구역 고기맛집]하남돼지집 서울대입구역점 솔직후기!!! , href : https://blog.naver.com/gorani_9?Redirect=Log&logNo=221